# ARPA Lombardia - Meteorological Stations Data Request and Processing Notebook

Author: Emanuele Capizzi - Email: emanuele.capizzi@polimi.it

This notebook allows to access to ARPA Lombardia meteorological stations using the datasets freely available on [Open Data Regione Lombardia Catalog](https://dati.lombardia.it/). This notebook allowed to test functionalities to be used in this QGIS plugin [ARPA Weather](https://github.com/gisgeolab/ARPA_Weather_plugin).

## Datasets organization and description

Firstly, it is necessary to understand the organization of the available datasets.<br>
The dataset that will be used in this notebook are:
1. Sensors information
2. Time-series measured by each sensor

By merging this datasets on the Sensor ID column it is possible to combine the information.

### 1. Sensors Information

The information about each sensor in ARPA's weather monitoring network (~ 1200/1300 sensors) are available at the following can be obtained directly from the API (Application Programming Interface), allowing a systematic way to request the data from the catalog. These information are provided separately from the sensors measured time-series.<br> 
<br> 
Link to the resource: https://www.dati.lombardia.it/Ambiente/Stazioni-Meteorologiche/nf78-nj6b


The sensors information accessible through this portal include: <br>

| Information description | Column Name |
| :---: | :---: |
| ID of the sensor | idsensore |
| Type of the Sensor | tipologia |
| Measure Unit | unit_dimisura |
| Station ID (multiple sensors can be available at the same station) | idstazione |
| Name of the Station | nomestazione |
| Height of the station (m) | quota |
| Start date of the time-series | datastart |
| Whether the sensor is historical | storico |
| WGS84 UTM 32N North Coordinate | cgb_nord |
| WGS84 UTM 32N East Coordinate | cgb_est |
| Longitude | lng |
| Latitude | lat |
| Dictionary containing latitude and longitude | location |
| Eventual end date of the time-series, if the sensor is not measuring | datastop |

Specifically, the following  are the types of sensors available in the ARPA Meteorological Network:<br>

| Sensor Type | Sensor Name | Measure Unit |
| :---: | :---: | :---: |
| Rainfall | Precipitazione | mm |
| Temperature | Temperatura | °C |
| Relative Humidity | Umidità Relativa | % |
| Global Solar Radiation | Radiazione Globale | W/m<sup>2</sup> |
| Wind Direction | Direzione Vento | Degrees North (°) |
| Wind Speed | Velocità Vento | m/s |
| Hydrometric Level | Livello Idrometrico | cm |
| Snow Depth | Altezza Neve | cm |

### 2. Measured Time-series

Subsequently, it is possible to request the time-series for each sensor. The temporal frequency of the measurement may vary depending on the sensor.

<div class="alert alert-block alert-warning">  
<b>For accessing such data, it should be pointed out that:</b> 
    <br><li>The time-series obtained from the API are updated regularly and can be accessed in real-time, but only from the beginning of the current month.
    <br><li>For historical data (past months and years), it's required to download as a zip folder containing CSV files containing yearly observations.
</div>

<div class="alert alert-block alert-success"> This notebook allows to handles it automatically, requesting data both from API or downloading the CSV files.
</div>

Link to the resource (API) containing sensors data: https://www.dati.lombardia.it/Ambiente/Dati-sensori-meteo/i95f-5avh <br>
<br>
The following table describe the time-series data obtainable through the API or the CSV files:<br>

| Information description | Column Name | Notes |
| :---: | :---: | :---: |
| ID of the sensor | IdSensore | Corresponding to the ID available in the sensors information table |
| Date of observation | Data | - |
| Observation value | Valore | LEGEND: -9999 = missing data / 888, 8888 = variable wind direction / 777, 7777 = quiet (for wind direction only) |
| ID of the Operator | IdOperatore | LEGEND: 1: Mean value / 3: Maximum Value / 4: Valore cumulated value (for rain) |
| Data status | Stato | LEGEND: VA, VV = Validated data / NA, NV, NC = Invalid data / NI = Uncertain data / ND = Data not available |


### Notes about the notebook
The functions used in this notebook are stored in a separated .py file that is imported at the beginning of the notebook, in order to keep the notebook as clean as possible.<br>
In order to see the widgets it may be necessary to update ipywidgets library or to use the following command in the Command Line: ```jupyter nbextension enable --py widgetsnbextension```
---

## Access to ARPA Lombardia data through Socrata API
### 1. Socrata API
The [Socrata Open Data API](https://dev.socrata.com/) allows you to programmatically access a wealth of open data resources from governments, non-profits, and NGOs around the world. <br>

<div class="alert alert-block alert-warning">  
<b>For accessing such data, it should be pointed out that:</b> 
While it is possible to perform simple unauthenticated queries against the Socrata Open Data API without making use of an application token, you’ll receive much higher throttling limits if you include an application token in your requests.
    <b>You can try to request data without using a Token and check if it works, but in any case the procedure to obtain a token is described.</b>
</div>

### 2. Sodapy
In this notebook [sodapy](https://github.com/xmunoz/sodapy) Python library is used. Sodapy is a python client for the Socrata Open Data API.

### 3. How to obtain a token
 - In order to get a token it is necessary to open [Open Data Lombardia](https://dati.lombardia.it/) website.
 - Subscribe to the website and go to your profile settings by clicking on your name in the upper-right corner. Then click on  `Il mio profilo ` tab.
 - Once your are on your profile, click on the  `Pen symbol` near the  `Il tuo profilo`, as shown here:<br>
 <br>
 <img src="./img/ARPA_API.png" style="display: block; margin: auto";/><br>
 <br>
- Modify your profile and open the  `Opzioni per lo sviluppatore` tab. Create a new  `App Token` to be be used by clicking  `Crea una nuova applicazione`, as shown: <br> 
 <br>
 <img src="./img/developer_settings.png"style="display: block; margin: auto";/><br>
 <br>
 
- Create your Token by inserting all the required information, as shown:<br> 
 <br>
 <img src="./img/app_token_modify.png" width="400px"  style="display: block; margin: auto";/><br>
 <br>

- Finally the token will be available in the  `Token App` table, as shown:<br> 
 <br>
 <img src="./img/final_token.png" width="900px"  style="display: block; margin: auto";/><br>
 <br>


<div class="alert alert-block alert-success">  
<b><span>&#x2714;</span></b> Now you can copy and paste the token to be used for accessing the Socrata API.
</div>

## Import libraries
The most relevant libraries to be installed for this notebook are the following:
- sodapy
- pandas
- dask
- ipywidgets
- matplotlib
- seaborn
- geopandas

In [ ]:
# Filter warnings
import warnings
warnings.filterwarnings('ignore')

# Libraries
from sodapy import Socrata
import pandas as pd
import dask.dataframe as dd
import numpy as np
import ipywidgets as widgets
import geopandas as gpd 
import requests
from datetime import datetime, timedelta
from io import BytesIO
from zipfile import ZipFile
import os
import time

# Plotting 
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator)
import matplotlib.dates as mdates
import seaborn as sns

# Import functions and set auto-reload
import ARPA_functions as f
%load_ext autoreload
%autoreload 2

## Create Client Connection

The following function will create a client connection to ARPA API. It will use the token if it's provided by the user, otherwise it will proceed without using the token. <b> Meteo data can be requested withouth Token for now, withouth any evident limitation. This step can be skipped. </b>

In [ ]:
token_text_w = widgets.Text(
    placeholder='Enter token here',
    description='Enter token (if available):',
    disabled=False,
    style= {'description_width': 'initial'},
    layout = widgets.Layout(width='400px'))
token_text_w

In [ ]:
arpa_token = token_text_w.value  # Use empty string to use the API without token
print(arpa_token)
client = f.connect_ARPA_api(arpa_token)

In [ ]:
stationsId = "nf78-nj6b" # Select meteo stations dataset containing positions and information about sensors
sensors_info = client.get_all(stationsId)

## Retrieve sensors information

As explained at the beginning of the notebook, it is necessary to retrieve all sensors information. The following function allows to obtain and print them:

In [ ]:
sensors_df = f.ARPA_sensors_info(client)
sensors_df.head()

You can select the sensor type from the following list:

In [ ]:
unique_sensors_list = sensors_df['tipologia'].unique()
sw = widgets.Dropdown(
    options=unique_sensors_list,
    value='Temperatura',
    description='Sensor type:')
sw

In [ ]:
sensor_sel = sw.value # Store selected sensor value
sensor_sel

Create a list of sensors that belong to the selected sensor type:

In [ ]:
sensors_list = (sensors_df.loc[sensors_df['tipologia'] == sensor_sel]).idsensore.tolist()  #& (sensors_df['storico'] == storic_data)
print(("Selected sensor: {sel}").format(sel=sensor_sel))
print(("Number of selected sensor: {sens_len}").format(sens_len=len(sensors_list)))

----

## Check API Time-Series Availability

The following function is used to obtain minimum and maximum date of the corresponding time-series available in the ARPA API.

In [ ]:
start_date_API, end_date_API = f.req_ARPA_start_end_date_API(client)

In [ ]:
api_start_limit = datetime(datetime.today().year, datetime.today().month, 1)
api_start_limit

## Define Date Range

In this section is possible to select the time range for data request. 

<div class="alert alert-block alert-warning">  
<li> Only dates within the same year can be selected.
<li>  If dates from previous month are requested the CSV file of the corresponding file will be downloaded and used for processing.
<li> The CSV are available from 2013 onwards.
</div>

In [ ]:
# Create a date picker
start_picker = widgets.DatetimePicker(description='Start Date:')
end_picker = widgets.DatetimePicker(description='End Date:')

In [ ]:
# Display the date and time pickers
widgets.VBox([start_picker, end_picker])

In [ ]:
start_date = start_picker.value.replace(tzinfo=None)
end_date = end_picker.value.replace(tzinfo=None)

# Check if dates are ok
year, start_date, end_date = f.check_dates(start_date, end_date)
print("Year:", year,"/ Start date:", start_date,"/ End date:", end_date)

---

## Download and process observations from API or csv files

<div class="alert alert-block alert-warning">  
If you see a limitation on data download you can launch the notebook using the following command: 
</div>

In [ ]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

## Download and process observations from API or csv files

The following code block will check if the chosen start date is before the start date available in the API.
If if the date is before that date, the CSV file corresponding to the selected year will be downloaded and processed.
For example, if the current  date is 20 February 2023 and you request data from 15 January 2023 to 15 February 2023:
- the 2023 CSV data will be downloaded and processed up to 31 January 2023.
- if you need the data from 01 February 2023 onwards, you need to set it as start date in order to request the data from the API.

In [ ]:
#If the chosen start date is before the start date available in the API -> use csv data
if start_date < api_start_limit:
    print("Requesting CSV")
    sensors_values = f.download_extract_csv_from_year(str(year)) #download the csv corresponding to the selected year
    csv_file = str(year)+'.csv'
    sensors_values = f.process_ARPA_csv(csv_file, start_date, end_date, sensors_list) #process csv file with dask
    
#If the chosen start date is equal or after the start date of API -> request data from API
elif start_date >= api_start_limit:
    print("Requesting from API")
    sensors_values = f.req_ARPA_data_API(client, start_date, end_date, sensors_list) #request data from ARPA API

In [ ]:
sensors_values

## Filter out outliers
Some filtering functions can be applied to remove invalid values and outlier, such as:
- Interquantile range (IQR): Outliers are defined as observations that fall below Q1 − 1.5 IQR or above Q3 + 1.5 IQR.
- Z-Score: calculate the Z-Score for the observations and remove those below a given threshold. (normally distributed data)

Select the method to be used:

In [ ]:
outlier_methods = ['iqr', 'zscore'] # List of methods for removing outliers
checkboxes = [widgets.Checkbox(value=False, description=label) for label in outlier_methods]
output = widgets.VBox(children=checkboxes)
display(output)

In [ ]:
selected_data = []
for i in range(0, len(checkboxes)):
    if checkboxes[i].value == True:
        selected_data = selected_data + [checkboxes[i].description]
print(selected_data)
if 'iqr' in selected_data:
    sensors_values = sensors_values.groupby('idsensore').apply(f.outlier_filter_iqr)
    print('Removed outliers using IQR')
if 'zscore' in selected_data:
    sensors_values = f.outlier_filter_zscore(sensors_values, sensors_list)
    print('Removed outliers using zscore')

## Data aggregation

Function to aggregate the data in the dataframe saved in the `sensors_values` dataframe.
Must provide:
- dataframe containing idsensore (int), data (datetime), valore (float).
- Temporal aggregation: hour (H), day (D), week (W), month (M), year (Y).

The statistics calculated are:
- Mode, count $\rightarrow$ for wind direction (since it's expressed in North Degrees).
- Mean, min, max, std, count $\rightarrow$ for all the remaining sensors.


Use the function to aggregate and group the data. You can pass the value obtained from the API or the values obtaines from the csv.

Choose the temporal aggregation:

In [ ]:
tagg_w = widgets.ToggleButtons(
    options=['H', 'D', 'W', 'M', 'Y'],
    description='Period aggregation:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Hour', 'Day', 'Week', 'Month', 'Year'])
tagg_w

In [ ]:
tagg = tagg_w.value

if sensor_sel != 'Direzione Vento':
    sensor_test_agg = f.aggregate_group_data(sensors_values, tagg)
if sensor_sel == 'Direzione Vento':
    sensor_test_agg = f.aggregate_group_data_wind_dir(sensors_values, tagg)

In [ ]:
sensor_test_agg

## Join sensors information and time series 
Once the dataframes are created is possible to merge the information.

In [ ]:
merged_df = pd.merge(sensor_test_agg, sensors_df, on='idsensore')
measure_unit = merged_df['unit_dimisura'].unique()[0]
print("Measure unit: " + measure_unit)
merged_df.head()

## Plotting

In [ ]:
if sensor_sel == 'Direzione Vento':
    stats_list = ['mode', 'count']
    default_value = 'mode'
if sensor_sel != 'Direzione Vento':
    stats_list = ['mean', 'max', 'min', 'std', 'count']
    default_value = 'mean'

In [ ]:
stat_w = widgets.Dropdown(
    options=stats_list,
    value= default_value,
    description='Value:')
stat_w

In [ ]:
stat_sel = stat_w.value

### Plot all sensors

In [ ]:
# Create a new figure
fig, ax = plt.subplots(figsize=(15, 10))
ax.set_title('ARPA Lombardia Sensors - ' + sensor_sel, fontdict={'fontsize': 10})
ax.set_ylabel(measure_unit, fontdict={'fontsize': 10})
ax.set_xlabel('Date', fontdict={'fontsize': 10})

# Iterate over the sensor IDs
for sensor_id in sensors_list:
    # Get the data for the current sensor
    sensor_data = merged_df[(merged_df['idsensore'] == sensor_id)]  # add & (merged_df['provincia'] == 'MI') in the square brackets if want to filter on Milan
    # Plot the time series for the sensor
    ax.plot(sensor_data['data'], sensor_data[stat_sel], label=sensor_id)


# Show the plot
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))
plt.gcf().autofmt_xdate()
plt.xticks(rotation=90)
plt.show()

### Check highest and lowest values

Select the sensor that provide the lowest value in the dataframe to check if there are possible errors:

In [ ]:
min_index = merged_df[stat_sel].idxmin()
idsensor_min = merged_df.loc[min_index].idsensore
sel_sensor_min = merged_df.loc[merged_df['idsensore']==idsensor_min]
data_min = merged_df.loc[min_index]['data']
merged_df.loc[min_index]

Select the sensor that provide the highest value in the dataframe:

In [ ]:
max_index = merged_df[stat_sel].idxmax()
idsensor_max = merged_df.loc[max_index].idsensore
sel_sensor_max = merged_df.loc[merged_df['idsensore']==idsensor_max]
data_max = merged_df.loc[max_index]['data']
merged_df.loc[max_index]

Plot min and max selected sensors:

In [ ]:
plt.figure(figsize=(15,8))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator())

plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))
plt.gcf().autofmt_xdate()
plt.xticks(rotation=90)
plt.plot(sel_sensor_min['data'],sel_sensor_min[stat_sel], '-',  color="tab:blue", label=("Min time-series - {nome} {quota} m").format(nome=sel_sensor_min["nomestazione"].unique()[0], quota=sel_sensor_min["quota"].unique()[0]) + " - " + stat_sel)
plt.plot(sel_sensor_max['data'],sel_sensor_max[stat_sel], '-',  color="tab:red", label=("Max time-series - {nome} {quota} m").format(nome=sel_sensor_max["nomestazione"].unique()[0], quota=sel_sensor_max["quota"].unique()[0]) + " - " + stat_sel)
plt.legend()

### Box Plot

In [ ]:
my_pal = {0: "tab:red", 1: "tab:blue"}
box = sns.boxplot(data=[sel_sensor_max[stat_sel], sel_sensor_min[stat_sel]], orient="v", palette=my_pal)
plt.xticks([0, 1],["Max time-series","Min time-series"])
box.annotate(("ID: {id}, Quota: {quota} m\n {nome}\n {stat}").format(id=sel_sensor_max["idsensore"].unique()[0],
             quota=sel_sensor_max["quota"].unique()[0], nome=sel_sensor_max["nomestazione"].unique()[0], stat=stat_sel),
             xy=(0, sel_sensor_max[stat_sel].max()+1),
             horizontalalignment='center',
             bbox=dict(boxstyle="round", fc="0.8"))
box.annotate(("ID: {id}, Quota: {quota} m\n {nome}\n {stat}").format(id=sel_sensor_min["idsensore"].unique()[0],
             quota=sel_sensor_min["quota"].unique()[0], nome=sel_sensor_min["nomestazione"].unique()[0], stat=stat_sel),
             xy=(1, sel_sensor_max[stat_sel].max()+1),
             horizontalalignment='center',
             bbox=dict(boxstyle="round", fc="0.8"))

### Map

Import Lombardy and CMM vector files:

In [ ]:
lombardy_gdf = gpd.read_file('lombardia_boundary.gpkg')
cmm_gdf = gpd.read_file('CMM.gpkg')

Convert to Geodataframe:

In [ ]:
gdf = gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(merged_df.cgb_est, merged_df.cgb_nord), crs="EPSG:32632")

In [ ]:
gdf.head()

Filter on the dates corresponding to the max and min values found before:

In [ ]:
min_date = merged_df.loc[min_index].data
max_date = merged_df.loc[max_index].data
min_date, max_date

In [ ]:
# Get the gdf corresponding with the date where the maximum and minimum values are observed
max_gdf = gdf.loc[merged_df['data'] == max_date]
min_gdf = gdf.loc[merged_df['data'] == min_date]

# Get the point (positions) where the minimum and maximum are observed
min_point = min_gdf.loc[min_gdf['idsensore'] == idsensor_min]
max_point = max_gdf.loc[max_gdf['idsensore'] == idsensor_max]

In [ ]:
fig, ax = plt.subplots(figsize = (9,9))
min_gdf.plot(ax=ax, markersize=20, column=stat_sel, legend=True, cmap='coolwarm', legend_kwds={'label': measure_unit ,'orientation': "vertical"});
lombardy_gdf.boundary.plot(ax=ax);
cmm_gdf.boundary.plot(ax=ax, edgecolor='orange');
min_point.plot(ax=ax, marker='o', markersize=400, color='black',facecolors='none')
ax.set_title(("ID: {id} - Quota: {quota} m - {nome} T°: {minT:0.2f}\n Data: {data} - Min - ARPA Lombardia Sensors - {sensor_sel}").format(id=sel_sensor_min["idsensore"].unique()[0],
             quota=sel_sensor_min["quota"].unique()[0], nome=sel_sensor_min["nomestazione"].unique()[0], minT=sel_sensor_min[stat_sel].min(),
             data=data_min.strftime("%Y-%m-%d"), sensor_sel=sensor_sel), fontdict = {'fontsize': 10})
ax.set_ylabel('North [m]',fontdict = {'fontsize': 10})
ax.set_xlabel('East [m]',fontdict = {'fontsize': 10})

In [ ]:
fig, ax = plt.subplots(figsize = (9,9))
max_gdf.plot(ax=ax, markersize=20, column=stat_sel, legend=True, cmap='coolwarm', legend_kwds={'label': measure_unit ,'orientation': "vertical"});
lombardy_gdf.boundary.plot(ax=ax);
cmm_gdf.boundary.plot(ax=ax, edgecolor='orange');
max_point.plot(ax=ax, marker='o', markersize=400, color='black',facecolors='none')
ax.set_title(("ID: {id} - Quota: {quota} m -  {nome} T°: {maxT:0.2f}\n Data: {data} - Max - ARPA Lombardia Sensors - {sensor_sel}").format(id=sel_sensor_max["idsensore"].unique()[0],
             quota=sel_sensor_max["quota"].unique()[0], nome=sel_sensor_max["nomestazione"].unique()[0], maxT=sel_sensor_max[stat_sel].max(),
             data=data_max.strftime("%Y-%m-%d"), sensor_sel=sensor_sel), fontdict = {'fontsize': 10})

ax.set_ylabel('North [m]',fontdict = {'fontsize': 10})
ax.set_xlabel('East [m]',fontdict = {'fontsize': 10})

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
max_gdf.plot(ax=ax, markersize=20, column='quota', legend=True, cmap='coolwarm', legend_kwds={'label': 'm' ,'orientation': "vertical"});
lombardy_gdf.boundary.plot(ax=ax);
cmm_gdf.boundary.plot(ax=ax, edgecolor='orange');
ax.set_title('ARPA Lombardia Sensors - Quota',fontdict = {'fontsize': 10})
ax.set_ylabel('North [m]',fontdict = {'fontsize': 10})
ax.set_xlabel('East [m]',fontdict = {'fontsize': 10})

In [ ]:
gdf_test = gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(merged_df.lng, merged_df.lat), crs="EPSG:4326")

In [ ]:
gdf_test.dtypes

In [ ]:
gdf_test.head()

In [ ]:
min_gdf["lat"] = [float(str(i).replace(",", "")) for i in min_gdf["lat"]]
min_gdf["lng"] = [float(str(i).replace(",", "")) for i in min_gdf["lng"]]

In [ ]:
max_gdf["lat"] = [float(str(i).replace(",", "")) for i in max_gdf["lat"]]
max_gdf["lng"] = [float(str(i).replace(",", "")) for i in max_gdf["lng"]]

In [ ]:
#using plotly for an animated choropleth map
import plotly.express as px
fig = px.scatter_mapbox(min_gdf,
                        lat="lat",
                        lon="lng",
                        hover_name="idsensore",
                        hover_data=["nomestazione","provincia",stat_sel],
                        color=stat_sel,
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        zoom=7,
                        height=600,
                        size='lat',
                        size_max=12,
                        opacity=1,
                        width=900)
fig.update_layout(mapbox_style='carto-darkmatter')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
fig = px.scatter_mapbox(max_gdf,
                        lat="lat",
                        lon="lng",
                        hover_name="idsensore",
                        hover_data=["nomestazione","provincia",stat_sel],
                        color=stat_sel,
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        zoom=7,
                        height=600,
                        size='lat',
                        size_max=12,
                        opacity=1,
                        width=900)
fig.update_layout(mapbox_style='carto-darkmatter')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()